In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dev.txt to dev.txt
Saving embs_train.kv to embs_train.kv
Saving svector.py to svector.py
Saving test.txt to test.txt
Saving train.py to train.py
Saving train.txt to train.txt
User uploaded file "dev.txt" with length 114951 bytes
User uploaded file "embs_train.kv" with length 17618313 bytes
User uploaded file "svector.py" with length 1336 bytes
User uploaded file "test.txt" with length 115200 bytes
User uploaded file "train.py" with length 1405 bytes
User uploaded file "train.txt" with length 918071 bytes


In [4]:
from gensim.models import KeyedVectors
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from svector import svector
import time

In [44]:
wv = KeyedVectors.load('embs_train.kv', mmap='r')

def read_from(textfile):
    for line in open(textfile):
        label, words = line.strip().split("\t")
        yield (1 if label=="+" else -1, words.split())

In [46]:
# @markdown Part 2.1.3

In [ ]:
def KNNsentenceEmbedding():
    trainfile= 'train.txt'
    devFile='dev.txt'
    sentences=[]
    devSentences=[]
    labels=[]
    devLabels=[]
    for j, (label, words) in enumerate(read_from(trainfile), 1):
        sentence = np.zeros((300,), dtype=float)
        count = 0
        for i in range(len(words)):
            try:
                sentence = sentence + wv[words[i]]
                count += 1
            except:
                pass
        sentence = sentence / count
        labels.append(label)
        sentences.append(sentence)
    for j, (label, words) in enumerate(read_from(devFile), 1):
        sentence = np.zeros((300,), dtype=float)
        count = 0
        for i in range(len(words)):
            try:
                sentence = sentence + wv[words[i]]
                count += 1
            except:
                pass
        if count>0:
            sentence = sentence / count
            devLabels.append(label)
            devSentences.append(sentence)
    for k in range(1,101,2):
        neigh = KNeighborsClassifier(n_neighbors=k)
        neigh.fit(sentences, labels)
        devCorrectCount = 0
        for i in range(len(devSentences)):
            devPrediction=neigh.predict([devSentences[i]])
            if devLabels[i]==devPrediction:
                devCorrectCount=devCorrectCount+1
        devPercentCorrect=devCorrectCount/len(devSentences)
        print("k=",k, " dev error rate: ", round((1-devPercentCorrect)*100,1), " (+:", round(devPercentCorrect*100,1), ")")


In [ ]:
KNNsentenceEmbedding()

k= 1  dev error rate:  36.9  (+: 63.1 )
k= 3  dev error rate:  34.7  (+: 65.3 )
k= 5  dev error rate:  34.4  (+: 65.6 )
k= 7  dev error rate:  33.7  (+: 66.3 )
k= 9  dev error rate:  31.8  (+: 68.2 )
k= 11  dev error rate:  30.6  (+: 69.4 )
k= 13  dev error rate:  31.2  (+: 68.8 )
k= 15  dev error rate:  30.3  (+: 69.7 )
k= 17  dev error rate:  31.2  (+: 68.8 )
k= 19  dev error rate:  30.8  (+: 69.2 )
k= 21  dev error rate:  30.8  (+: 69.2 )
k= 23  dev error rate:  31.1  (+: 68.9 )
k= 25  dev error rate:  30.1  (+: 69.9 )
k= 27  dev error rate:  30.3  (+: 69.7 )
k= 29  dev error rate:  29.2  (+: 70.8 )
k= 31  dev error rate:  29.5  (+: 70.5 )
k= 33  dev error rate:  30.4  (+: 69.6 )
k= 35  dev error rate:  29.6  (+: 70.4 )
k= 37  dev error rate:  29.9  (+: 70.1 )
k= 39  dev error rate:  29.9  (+: 70.1 )
k= 41  dev error rate:  30.3  (+: 69.7 )
k= 43  dev error rate:  29.6  (+: 70.4 )
k= 45  dev error rate:  29.2  (+: 70.8 )
k= 47  dev error rate:  29.2  (+: 70.8 )
k= 49  dev error rate

In [ ]:
def KNNOneHot():
    trainfile= 'train.txt'
    devFile='dev.txt'
    sentences=[]
    devSentences=[]
    labels=[]
    devLabels=[]
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    #get all the words to fit
    allWords=[]
    for i, (label, words) in enumerate(read_from(trainfile), 1):
        for word in words:
            allWords.append(word)
    for i, (label, words) in enumerate(read_from(devFile), 1):
        for word in words:
            allWords.append(word)
    label_encoder = LabelEncoder()
    allWords=np.unique(allWords)
    print(len(allWords))
    label_encoder.fit(allWords)
    vocabLength=len(allWords)

    for j, (label, words) in enumerate(read_from(trainfile), 1):
        sentence = np.zeros((vocabLength,), dtype=float)
        labelEncoded = label_encoder.transform(words)
        sentence[labelEncoded] = 1
        labels.append(label)
        sentences.append(sentence)
    for j, (label, words) in enumerate(read_from(devFile), 1):
        sentence = np.zeros((vocabLength,), dtype=float)

        labelEncoded = label_encoder.transform(words)
        sentence[labelEncoded] = 1
        devLabels.append(label)
        devSentences.append(sentence)
    for k in range(1,101,2):
        neigh = KNeighborsClassifier(n_neighbors=k)
        neigh.fit(sentences, labels)
        devCorrectCount = 0
        for i in range(len(devSentences)):
            devPrediction=neigh.predict([devSentences[i]])
            if devLabels[i]==devPrediction:
                devCorrectCount=devCorrectCount+1
        devPercentCorrect=devCorrectCount/len(devSentences)
        print("k=",k, " dev error rate: ", round((1-devPercentCorrect)*100,1), " (+:", round(devPercentCorrect*100,1), ")")



In [ ]:
KNNOneHot()

16743
made sentences from train
made sentences from dev
k= 1  dev error rate:  40.9  (+: 59.1 )
k= 3  dev error rate:  42.0  (+: 58.0 )
k= 5  dev error rate:  40.5  (+: 59.5 )
k= 7  dev error rate:  40.6  (+: 59.4 )
k= 9  dev error rate:  39.6  (+: 60.4 )
k= 11  dev error rate:  39.8  (+: 60.2 )
k= 13  dev error rate:  39.9  (+: 60.1 )
k= 15  dev error rate:  39.3  (+: 60.7 )
k= 17  dev error rate:  40.6  (+: 59.4 )
k= 19  dev error rate:  41.4  (+: 58.6 )
k= 21  dev error rate:  42.4  (+: 57.6 )
k= 23  dev error rate:  40.5  (+: 59.5 )
k= 25  dev error rate:  40.8  (+: 59.2 )
k= 27  dev error rate:  40.4  (+: 59.6 )
k= 29  dev error rate:  41.7  (+: 58.3 )
k= 31  dev error rate:  41.2  (+: 58.8 )
k= 33  dev error rate:  40.3  (+: 59.7 )
k= 35  dev error rate:  41.8  (+: 58.2 )
k= 37  dev error rate:  41.1  (+: 58.9 )
k= 39  dev error rate:  40.8  (+: 59.2 )
k= 41  dev error rate:  40.1  (+: 59.9 )
k= 43  dev error rate:  40.8  (+: 59.2 )
k= 45  dev error rate:  41.6  (+: 58.4 )
k= 47 

In [28]:
# @title Part 2.2 Reimplement Perceptron with Sentence Embedding


def BasicPerceptron():
    print("Basic Perceptron")
    epochs=10
    trainfile= 'train.txt'
    devfile='dev.txt'
    best_err = 1.
    model = np.zeros((300,), dtype=float)  # Initialize the model
    for it in range(1, epochs+1):
        updates = 0
        total_train = 0

        # Training Phase
        for i, (label, words) in enumerate(read_from(trainfile), 1):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                try:
                    sentence += wv[word]  # Add word vector
                    count += 1
                except KeyError:
                    pass  # Ignore words not in the embedding
            if count > 0:
                sentence /= count  # Average the word vectors

            if label * (model.dot(sentence)) <= 0:
                updates += 1
                model += label * sentence
            total_train += 1

        # Evaluation Phase
        total_dev, err_dev = 0, 0
        for label, words in read_from(devfile):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                try:
                    sentence += wv[word]
                    count += 1
                except KeyError:
                    pass
            if count > 0:
                sentence /= count

            err_dev += label * (model.dot(sentence)) <= 0
            total_dev += 1

        dev_err = err_dev / total_dev
        best_err = min(best_err, dev_err)
        print(f"Epoch {it}, Update {updates / total_train * 100:.2f}%, Dev Error {dev_err * 100:.2f}%")

BasicPerceptron()

Basic Perceptron
Epoch 1, Update 31.06%, Dev Error 37.40%
Epoch 2, Update 29.54%, Dev Error 35.40%
Epoch 3, Update 29.76%, Dev Error 33.20%
Epoch 4, Update 29.14%, Dev Error 40.00%
Epoch 5, Update 29.70%, Dev Error 35.20%
Epoch 6, Update 29.36%, Dev Error 40.40%
Epoch 7, Update 29.40%, Dev Error 38.40%
Epoch 8, Update 29.44%, Dev Error 42.50%
Epoch 9, Update 29.11%, Dev Error 39.00%
Epoch 10, Update 29.15%, Dev Error 39.20%


In [33]:

def AveragedPerceptron():
    print("Averaged Perceptron")
    epochs = 10
    trainfile = 'train.txt'
    devfile = 'dev.txt'
    model = np.zeros((300,), dtype=float)
    wa = np.zeros((300,), dtype=float)  # For averaged perceptron
    c = 0  # Update counter

    for it in range(1, epochs + 1):
        updates = 0

        for i, (label, words) in enumerate(read_from(trainfile), 1):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                try:
                    sentence += wv[word]
                    count += 1
                except KeyError:
                    pass
            if count > 0:
                sentence /= count

            if label * (model.dot(sentence)) <= 0:
                updates += 1
                model += label * sentence
                wa += c * label * sentence
            c += 1

        avg_model = model - wa / c

        # Evaluation Phase
        total_dev, err_dev = 0, 0
        for label, words in read_from(devfile):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                try:
                    sentence += wv[word]
                    count += 1
                except KeyError:
                    pass
            if count > 0:
                sentence /= count

            err_dev += label * (avg_model.dot(sentence)) <= 0
            total_dev += 1

        dev_err = err_dev / total_dev
        print(f"Epoch {it}, Update {updates / i * 100:.2f}%, Dev Error {dev_err * 100:.2f}%")


AveragedPerceptron()

Averaged Perceptron
Epoch 1, Update 31.06%, Dev Error 24.90%
Epoch 2, Update 29.54%, Dev Error 23.90%
Epoch 3, Update 29.76%, Dev Error 24.30%
Epoch 4, Update 29.14%, Dev Error 24.10%
Epoch 5, Update 29.70%, Dev Error 24.20%
Epoch 6, Update 29.36%, Dev Error 23.90%
Epoch 7, Update 29.40%, Dev Error 23.60%
Epoch 8, Update 29.44%, Dev Error 23.80%
Epoch 9, Update 29.11%, Dev Error 24.10%
Epoch 10, Update 29.15%, Dev Error 24.40%


In [34]:
from collections import Counter

def OneCountPerceptron():
    print("One Count Perceptron")
    epochs = 10
    trainfile = 'train.txt'
    devfile = 'dev.txt'

    # prune one-count words
    def prune_one_count_words(textfile):
        word_counts = Counter()
        for _, words in read_from(textfile):
            word_counts.update(words)
        return set(word for word, count in word_counts.items() if count > 1)

    def test_model(devfile, model):
        tot, err = 0, 0
        for i, (label, words) in enumerate(read_from(devfile), 1):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                try:
                    sentence += wv[word]
                    count += 1
                except KeyError:
                    pass
            if count > 0:
                sentence /= count
            err += label * (model.dot(sentence)) <= 0
        return err / i

    # Pruning one-count words
    pruned_words = prune_one_count_words(trainfile)

    model = np.zeros((300,), dtype=float)
    wa = np.zeros((300,), dtype=float)
    c = 1

    # Training with pruned words
    for it in range(1, epochs + 1):
        updates = 0
        total = 0
        for i, (label, words) in enumerate(read_from(trainfile), 1):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                if word in pruned_words:
                    try:
                        sentence += wv[word]
                        count += 1
                    except KeyError:
                        pass
            if count > 0:
                sentence /= count

            if label * (model.dot(sentence)) <= 0:
                updates += 1
                model += label * sentence
                wa += label * c * sentence
            c += 1
            total = i

        averaged_model = model - wa / c

        dev_err = test_model(devfile, averaged_model)
        print(f"Epoch {it}, Updates {updates / total * 100:.2f}%, Dev Error {dev_err * 100:.1f}%")

OneCountPerceptron()

One Count Perceptron
Epoch 1, Updates 31.96%, Dev Error 25.7%
Epoch 2, Updates 30.44%, Dev Error 25.5%
Epoch 3, Updates 30.31%, Dev Error 25.6%
Epoch 4, Updates 29.93%, Dev Error 25.3%
Epoch 5, Updates 30.36%, Dev Error 25.5%
Epoch 6, Updates 30.74%, Dev Error 25.9%
Epoch 7, Updates 30.44%, Dev Error 25.5%
Epoch 8, Updates 30.14%, Dev Error 25.5%
Epoch 9, Updates 29.93%, Dev Error 25.3%
Epoch 10, Updates 30.23%, Dev Error 25.4%


In [40]:
import numpy as np
from sklearn.svm import SVC
from collections import Counter

def prune_low_count_words(textfile, min_count=2):
    word_counts = Counter()
    for _, words in read_from(textfile):
        word_counts.update(words)
    return set(word for word, count in word_counts.items() if count >= min_count)

def prepare_data(textfile, pruned_words):
    sentences = []
    labels = []
    for label, words in read_from(textfile):
        sentence = np.zeros((300,), dtype=float)
        count = 0
        for word in words:
            if word in pruned_words:
                try:
                    sentence += wv[word]
                    count += 1
                except KeyError:
                    pass
        if count > 0:
            sentence /= count
        sentences.append(sentence)
        labels.append(label)
    return np.array(sentences), np.array(labels)

def train_and_evaluate_SVM(trainfile, devfile):
    # Pruning low-count words
    pruned_words = prune_low_count_words(trainfile)

    # Preparing data
    train_sentences, train_labels = prepare_data(trainfile, pruned_words)
    dev_sentences, dev_labels = prepare_data(devfile, pruned_words)

    # Training SVM
    clf = SVC()
    clf.fit(train_sentences, train_labels)

    # Evaluating on development set
    dev_predictions = clf.predict(dev_sentences)
    dev_error_rate = np.mean(dev_predictions != dev_labels)
    print(f"Dev Error Rate: {dev_error_rate * 100:.2f}%")

start_time = time.time()
train_and_evaluate_SVM('train.txt', 'dev.txt')
end_time = time.time()
runtime = end_time - start_time

print(f"Running time: {runtime:.2f} seconds")

Dev Error Rate: 23.40%
Running time: 15.71 seconds


In [50]:
# @title Part4

def PredictBest():
    trainfile = 'train.txt'
    devfile = 'dev.txt'
    testfile = 'test.txt'
    outputfile = 'test.txt.predicted.label'
    epochs= 10
    model = np.zeros((300,), dtype=float)
    wa = np.zeros((300,), dtype=float)
    c = 1
    best_dev_err = 1.0
    # Training Phase
    for it in range(1, epochs + 1):
        for label, words in read_from(trainfile):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                try:
                    sentence += wv[word]
                    count += 1
                except KeyError:
                    pass
            if count > 0:
                sentence /= count

            if label * (model.dot(sentence)) <= 0:
                model += label * sentence
                wa += label * c * sentence
            c += 1

    avg_model = model - wa / c

    total_dev, err_dev = 0, 0
    for label, words in read_from(devfile):
            sentence = np.zeros((300,), dtype=float)
            count = 0
            for word in words:
                try:
                    sentence += wv[word]
                    count += 1
                except KeyError:
                    pass
            if count > 0:
                sentence /= count

            err_dev += label * (avg_model.dot(sentence)) <= 0
            total_dev += 1

    dev_err = err_dev / total_dev
    best_dev_err = min(best_dev_err, dev_err)

    print("best dev err", best_dev_err)

    # Prediction on test set
    predictions = []
    for _, words in read_from(testfile):
        sentence = np.zeros((300,), dtype=float)
        count = 0
        for word in words:
            try:
                sentence += wv[word]
                count += 1
            except KeyError:
                pass
        if count > 0:
            sentence /= count
        prediction = np.sign(avg_model.dot(sentence))
        predictions.append('+' if prediction == 1 else '-')

    # Writing predictions to file
    with open(outputfile, 'w') as f:
        for prediction in predictions:
            f.write(prediction + '\n')

    return outputfile

# File paths


# Train the model and predict on test set
PredictBest()


best dev err 0.244


'test.txt.predicted.label'

In [52]:
import pandas as pd
import numpy as np
test_data = pd.read_csv("test.txt", sep='\t', header=None)
predicted_data = pd.read_csv("test.txt.predicted.label", header=None)

In [53]:
test_data

,0,1
0,?,a seriocomic debut of extravagant promise by g...
1,?,while you have to admit it 's semi amusing to ...
2,?,"a boldly stroked , luridly coloured , uni dime..."
3,?,"an enormously entertaining movie , like nothin..."
4,?,"in his latest effort , storytelling , solondz ..."
...,...,...
995,?,the best thing i can say about this film is th...
996,?,wonder of wonders a teen movie with a humanist...
997,?,the wild thornberrys movie is pleasant enough ...
998,?,as a thoughtful and unflinching examination of...


In [54]:
predicted_data

,0
0,+
1,+
2,-
3,-
4,-
...,...
995,-
996,+
997,+
998,+


In [55]:
test_data = test_data.drop([0],axis=1)

In [56]:
test_data

,1
0,a seriocomic debut of extravagant promise by g...
1,while you have to admit it 's semi amusing to ...
2,"a boldly stroked , luridly coloured , uni dime..."
3,"an enormously entertaining movie , like nothin..."
4,"in his latest effort , storytelling , solondz ..."
...,...
995,the best thing i can say about this film is th...
996,wonder of wonders a teen movie with a humanist...
997,the wild thornberrys movie is pleasant enough ...
998,as a thoughtful and unflinching examination of...


In [61]:
df = pd.concat([predicted_data, test_data], axis=1)


In [65]:
df

,0,1
0,+,a seriocomic debut of extravagant promise by g...
1,+,while you have to admit it 's semi amusing to ...
2,-,"a boldly stroked , luridly coloured , uni dime..."
3,-,"an enormously entertaining movie , like nothin..."
4,-,"in his latest effort , storytelling , solondz ..."
...,...,...
995,-,the best thing i can say about this film is th...
996,+,wonder of wonders a teen movie with a humanist...
997,+,the wild thornberrys movie is pleasant enough ...
998,+,as a thoughtful and unflinching examination of...


In [66]:
df.to_csv('test.txt.predicted', sep = '\t', index = False)